# Imports and system definition

In [ ]:
import numpy as np
import scipy.linalg
import scipy.signal
import matplotlib.pyplot as plt

# State-space matrices from the combined model
A = np.array([
    [0, 1, 0],
    [-6.356, 0, 1.86],
    [6.35, 0, -33.2]
])

B = np.array([
    [0],
    [-2.49],
    [44.5]
])

C = np.array([
    [1, 0, 0],
    [0, 0, 1]
])

D = np.array([
    [0],
    [0]
])

# Create the continuous-time state-space system
system = scipy.signal.StateSpace(A, B, C, D)

print("State-space model defined successfully!")
print("\nA =\n", A)
print("\nB =\n", B)
print("\nC =\n", C)
print("\nD =\n", D)

# Controllability and Observability

- **Controllability**: A system is controllable if it is possible to move the system from any initial state to any desired final state in a finite time. We can check this by calculating the rank of the controllability matrix.
- **Observability**: A system is observable if it is possible to determine the internal state of the system by observing its outputs. We can check this by calculating the rank of the observability matrix.

In [ ]:
# Controllability
ctrb_matrix = np.hstack([B, A @ B, A @ A @ B])
ctrb_rank = np.linalg.matrix_rank(ctrb_matrix)

print(f"Controllability Matrix Rank: {ctrb_rank}")
if ctrb_rank == A.shape[0]:
    print("The system is controllable. ✅")
else:
    print("The system is not controllable. ❌")

# Observability
obsv_matrix = np.vstack([C, C @ A, C @ A @ A])
obsv_rank = np.linalg.matrix_rank(obsv_matrix)

print(f"\nObservability Matrix Rank: {obsv_rank}")
if obsv_rank == A.shape[0]:
    print("The system is observable. ✅")
else:
    print("The system is not observable. ❌")

# Controller Design

## Discretization

In [ ]:
# Discretize the system
dt_controller = 0.01  # Controller sample time
discrete_system = system.to_discrete(dt=dt_controller)
Ad = discrete_system.A
Bd = discrete_system.B

print("Discrete-time system matrices:")
print("\nAd =\n", Ad)
print("\nBd =\n", Bd)

## Tunning

In [ ]:
# Tune the controller

# Open Loop Simulation Without Disturbances

In [ ]:
# --- Simulation parameters ---
T_sim_ol = 0.6
dt_sim = 0.0001  # Time step
t_ol = np.arange(0, T_sim_ol, dt_sim)
x0_ol = np.array([0.5, 0, 0])  # Initial condition

# --- Simulation loop ---
x_ol = x0_ol
x_ol_history = [x0_ol]
for _ in t_ol[1:]:
    u_ol = np.array([0]) # No control input
    x_ol = Ad @ x_ol + Bd @ u_ol # Evolve the system
    x_ol_history.append(x_ol)

x_ol_history = np.array(x_ol_history)

# --- Plot the results ---
plt.figure(figsize=(12, 6))
plt.plot(t_ol, x_ol_history[:, 0], label='$\\theta$ (Body Angle)')
plt.plot(t_ol, x_ol_history[:, 1], label='$\\dot{\\theta}$ (Body Angular Velocity)')
plt.plot(t_ol, x_ol_history[:, 2], label='$\\dot{\\phi}$ (Wheel Angular Velocity)')
plt.title('Open-Loop Response (No Control, No Disturbances)')
plt.xlabel('Time (s)')
plt.ylabel('State')
plt.grid(True)
plt.legend()
plt.show()

# Closed Loop Simulation Without Disturbances

In [ ]:
# Simulation parameters
T_sim = 0.6
dt = 0.0001  # Time step
t = np.arange(0, T_sim, dt)
x0 = np.array([0.5, 0, 0])  # Initial condition (small tilt)

# Simulation loop
x = x0
x_history = [x0]
for _ in t[1:]:
    u = -K @ x
    x = Ad @ x + Bd @ u.flatten()
    x_history.append(x)

x_history = np.array(x_history)

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(t, x_history[:, 0], label='$\\theta$ (Body Angle)')
plt.plot(t, x_history[:, 1], label='$\\dot{\\theta}$ (Body Angular Velocity)')
plt.plot(t, x_history[:, 2], label='$\\dot{\\phi}$ (Wheel Angular Velocity)')
plt.title('Closed-Loop Control')
plt.xlabel('Time (s)')
plt.ylabel('State')
plt.grid(True)
plt.legend()
plt.show()

# Open Loop Simulation With Disturbances

In [ ]:
# --- Simulation parameters and disturbance generation ---
T_sim_dist_ol = 20
dt_dist = 0.0001
t_dist_ol = np.arange(0, T_sim_dist_ol, dt_dist)
x0_dist_ol = np.array([0.0, 0, 0]) # Start from a stable position

# Generate the same disturbance signal for a fair comparison
disturbance_magnitude = 0.04
disturbance_interval = 2.0 # seconds
num_samples_in_interval = int(disturbance_interval / dt_dist)
num_disturbance_steps = int(np.ceil(T_sim_dist_ol / disturbance_interval))
random_steps = disturbance_magnitude * np.random.randn(num_disturbance_steps)
slow_disturbance_ol = np.repeat(random_steps, num_samples_in_interval)
slow_disturbance_ol = slow_disturbance_ol[:len(t_dist_ol)]


# --- Simulation loop ---
x_dist_ol = x0_dist_ol
x_dist_ol_history = [x0_dist_ol]

for i, time in enumerate(t_dist_ol[1:]):
    # The only input is the disturbance itself
    u_dist_ol = np.array([slow_disturbance_ol[i]])
    
    # Update the state
    x_dist_ol = Ad @ x_dist_ol + Bd @ u_dist_ol
    x_dist_ol_history.append(x_dist_ol)

x_dist_ol_history = np.array(x_dist_ol_history)


# --- Plot the results ---
plt.figure(figsize=(12, 8))

# Plot 1: Body Angle State
ax1 = plt.subplot(2, 1, 1)
ax1.plot(t_dist_ol, x_dist_ol_history[:, 0], label='$\\theta$ (Body Angle)')
ax1.set_title('Open-Loop System Response to Disturbances')
ax1.set_ylabel('Angle (rad)')
ax1.grid(True)
ax1.legend()

# Plot 2: Disturbance Signal
ax2 = plt.subplot(2, 1, 2, sharex=ax1)
ax2.plot(t_dist_ol, slow_disturbance_ol, 'r-', label='Disturbance Input')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Disturbance')
ax2.grid(True)
ax2.legend()

plt.tight_layout()
plt.show()

# Closed Loop Simulation With Disturbances

In [ ]:
# --- Simulation parameters and disturbance generation ---
T_sim = 20
t = np.arange(0, T_sim, dt)
x0 = np.array([0.0, 0, 0]) # Start from a stable position

disturbance_magnitude = 0.04
disturbance_interval = 2.0 # seconds
num_samples_in_interval = int(disturbance_interval / dt)
num_disturbance_steps = int(np.ceil(T_sim / disturbance_interval))
random_steps = disturbance_magnitude * np.random.randn(num_disturbance_steps)
slow_disturbance = np.repeat(random_steps, num_samples_in_interval)
slow_disturbance = slow_disturbance[:len(t)]


# --- Simulation loop with control effort recording ---
x = x0
x_history = [x0]
u_history = []

for i, time in enumerate(t):
    # Calculate control signal
    u = -K @ x + slow_disturbance[i]

    # Store the control signal
    u_history.append(u)

    # Update the state
    if i < len(t) - 1: # Ensure we don't go past the end of the time array
        x = Ad @ x + Bd @ u.flatten()
        x_history.append(x)


x_history = np.array(x_history)
u_history = np.array(u_history).flatten() # Flatten for plotting


# --- Plot the results ---
plt.figure(figsize=(12, 10))

# Plot 1: Body Angle State
ax1 = plt.subplot(3, 1, 1)
ax1.plot(t, x_history[:, 0], label='$\\theta$ (Body Angle)')
ax1.set_title('System Response and Controller Effort with Disturbances')
ax1.set_ylabel('Angle (rad)')
ax1.grid(True)
ax1.legend()

# Plot 2: Disturbance Signal
ax2 = plt.subplot(3, 1, 2, sharex=ax1)
ax2.plot(t, slow_disturbance, 'r-', label='Disturbance')
ax2.set_ylabel('Disturbance')
ax2.grid(True)
ax2.legend()

# Plot 3: Controller Effort
ax3 = plt.subplot(3, 1, 3, sharex=ax1)
ax3.plot(t, u_history, 'g-', label='Control Signal (u)')
ax3.set_xlabel('Time (s)')
ax3.set_ylabel('Controller Effort')
ax3.grid(True)
ax3.legend()

plt.tight_layout()
plt.show()